### agent-based architecturem

autonomously decide and take actions without any human intervention

The original framework was ReAct, allowing an LLM to create observations after taking actions via tools. 

In [ ]:
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action

## >>> Implementation Steps

For this project, we will be using OpenAI function calling and Langchain LCEL to build the Agent.

### Setup OpenAI key

In [14]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_qdrant import Qdrant
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from dotenv import load_dotenv
import os

In [9]:
import  nest_asyncio
nest_asyncio.apply()

In [10]:
env_path = '/Users/andishehtavakoli/Documents/github-project/llm-apps/autonomous-agents/.env'


# Load environment variables from the specified .env file
load_dotenv(dotenv_path=env_path)
# Get the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Step 0: Load Data

In [24]:
from pathlib import Path    

In [32]:
BASE_DIR = Path('.').resolve().parent

In [34]:
BASE_DIR

PosixPath('/Users/andishehtavakoli/Documents/github-project/llm-apps/autonomous-agents')

In [39]:
# pip install pypdf

In [35]:
mental_disorder_path = [
    BASE_DIR / "data//generalized_anxiety_disorder.pdf",
    BASE_DIR / "data/ptsd_open_source.pdf",
    BASE_DIR / "data/schizophrenia.pdf",
  
]

In [38]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(BASE_DIR / "data")

docs = loader.load()

docs[0]

Document(metadata={'source': '/Users/andishehtavakoli/Documents/github-project/llm-apps/autonomous-agents/data/schizophrenia.pdf', 'page': 0}, page_content='National Institute\nof Mental Health\nSchizophrenia')

 # Step 1: Define Tools

1. **scrape_books** : Scrape books and book reviews from google and amazon
2. **find_relevant_books**: Retrieves relevant books based on a user query.
3. **create_topic_network**: Creates a visualization of topics in the books.
4. **qa**: Answers user's questions based on retrieved documents

 ### Step 2: Setting up the vector database

In [43]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
def create_db(documents):
    return Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name="my_documents",
        location=":memory:",
        force_recreate=False,
    )
db = create_db(docs)

/Users/andishehtavakoli/miniconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [42]:
session_state = {}

In [44]:
@tool
def find_relevant_books(user_query):
    """
    Return all relevant books based on user query.
    Important: This function should be called only for queries that require finding specific books.
    For general queries that do not require finding specific books, use other available functions.
    """
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.25}
    )
    relevant_docs = retriever.get_relevant_documents(user_query)
    session_state["relevant_docs"] = relevant_docs
    session_state["retriever"] = retriever
    return relevant_docs

llm = ChatOpenAI(
    model="gpt-4o", 
    temperature=0, 
    openai_api_key=os.getenv("OPEN_AI_KEY")
)
@tool
def qa(user_query):
    """
    Answer user questions based on the retrieved documents
    """
    retriever = session_state["retriever"]
    relevant_docs = session_state.get("relevant_docs")
    if relevant_docs is None:
        # If no documents are stored, retrieve them
        relevant_docs = retriever.get_relevant_documents(user_query)
        session_state["relevant_docs"] = relevant_docs
    
    # Create a chain to answer questions using stored documents
    qa = ConversationalRetrievalChain.from_llm(llm, retriever)
    chat_history = []
    result = qa(
        {"question": user_query, "chat_history": chat_history, "context": relevant_docs}
    )
    return result

When decorating these actions using @tool , the main agent will have access to a list of functions, their arguments and docstrings. This enables the agent to smartly choose the most relevant tool for the task.

we will store the relevant documents and the retriever in a globally defined dictionary session_state . This makes it easier for the agent to access this information.

# Step 2. Create the prompt

Now we set up the prompt with a system message, user message, and a MessagesPlaceholderthat allows the agent to store its intermediate steps:

In [45]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the prompt template
prompt_template = """
You are a helpful AI assistant specializing in answering questions 
related to books from users. Use retrieved relevant books to 
answer questions.
====================
{relevant_docs}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are helpful AI assistant. Use the following 
               template for your actions and observations."""
        ),
        ("user", prompt_template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

The scratchpad is where the agent will store all the intermediate results. For example, if the user asks to create a visualization of all the topics for the first Harry Potter book, the agent will first find the relevant book (the philosopher's stone), store the output in the scratchpad, then reason that it should call create_topic_network next.

# Step 3. Initialize the agent

In [46]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.tools import Tool, format_tool_to_openai_function

# These are custom functions for finding books, answering questions, and creating topic networks.
tools = [find_relevant_books, qa]
# OpenAI Function Formatting. This converts the tools into a format compatible with OpenAI's function calling feature.
functions = [format_tool_to_openai_function(f) for f in tools]
#This sets up the GPT-4o model with the defined functions.
model = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o",
).bind(functions=functions)

In [47]:
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

# Set up the agent chain.
# including assigning relevant documents and agent scratchpad, applying the prompt, running the model, and parsing the output.
agent_chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"]),
        relevant_docs=lambda x: "\n".join(
            str(doc) for doc in session_state.get("relevant_docs", [])
        ),
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)
# Set up a memory component to store conversation history.
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
    input_key="input",
    output_key="output",
)
# Initialize an agent with the agent and defined tools
# This combines all components into an executable agent that can process queries and maintain conversation context.
# With AgentExecutor, the agent is equipped with the tools and verbose output is enabled, allowing for detailed logging.
agent = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [48]:
agent.invoke(
    {
        "input": "what is the symptom of anxiety?"
    }
)



> Entering new AgentExecutor chain...
Anxiety symptoms can vary widely, but common symptoms include:

1. **Physical Symptoms:**
   - Increased heart rate
   - Rapid breathing or hyperventilation
   - Sweating
   - Trembling or shaking
   - Feeling weak or tired
   - Gastrointestinal problems
   - Headaches
   - Insomnia

2. **Emotional Symptoms:**
   - Feelings of nervousness or restlessness
   - A sense of impending danger, panic, or doom
   - Increased irritability
   - Difficulty concentrating or thinking about anything other than the present worry

3. **Behavioral Symptoms:**
   - Avoiding situations that trigger anxiety
   - Compulsive behaviors (in some anxiety disorders like OCD)
   - Social withdrawal

4. **Cognitive Symptoms:**
   - Excessive worry about everyday situations
   - Overthinking worst-case scenarios
   - Difficulty controlling worry

If you are looking for more detailed information or specific books on anxiety, I can help find relevant books for you. Would you l

{'input': 'what is the symptom of anxiety?',
 'chat_history': [HumanMessage(content='what is the symptom of anxiety?'),
  AIMessage(content='Anxiety symptoms can vary widely, but common symptoms include:\n\n1. **Physical Symptoms:**\n   - Increased heart rate\n   - Rapid breathing or hyperventilation\n   - Sweating\n   - Trembling or shaking\n   - Feeling weak or tired\n   - Gastrointestinal problems\n   - Headaches\n   - Insomnia\n\n2. **Emotional Symptoms:**\n   - Feelings of nervousness or restlessness\n   - A sense of impending danger, panic, or doom\n   - Increased irritability\n   - Difficulty concentrating or thinking about anything other than the present worry\n\n3. **Behavioral Symptoms:**\n   - Avoiding situations that trigger anxiety\n   - Compulsive behaviors (in some anxiety disorders like OCD)\n   - Social withdrawal\n\n4. **Cognitive Symptoms:**\n   - Excessive worry about everyday situations\n   - Overthinking worst-case scenarios\n   - Difficulty controlling worry\n\nI

#  Step 4. Creating the User Interface with Panel